In [2]:
import sys
import time

import cv2
from IPython.display import display, Image
%matplotlib inline

In [3]:
sys.path.append("../facenetLib")  # go to parent dir
import contributed.face as face

In [ ]:
class DetectorFace(object):
    model = '../models/20180402-114759.pb'
    save_point = '../models/one_shot_classifier.pkl'
    color =  (0, 255, 0)
    text_color=(255, 0, 0)
    rect_size = 2

    def __init__(self):
        self.image = None
        self.face_recognition = face.Recognition(DetectorFace.model,DetectorFace.save_point)

    def predict(self, frame):
        faces = self.face_recognition.identify(frame)

        if faces:
            for img in faces:
                face_bb = img.bounding_box.astype(int)
                pt1 = (face_bb[0], face_bb[1])
                pt2 = (face_bb[2], face_bb[3])
                cv2.rectangle(frame,pt1,pt2,DetectorFace.color, DetectorFace.rect_size)
                if img.name is not None:
                    print(img.embedding)
                    cv2.putText(frame,
                                img.name,
                                pt2,
                                cv2.FONT_HERSHEY_SIMPLEX,
                                1,
                                DetectorFace.text_color,
                                thickness=2,
                                lineType=2)

        return frame

In [ ]:
detecter = DetectorFace()

video_path = '../data/wisenet_dataset/video/set_1/video1_5.avi'

video = cv2.VideoCapture(video_path)
display_handle = display(None, display_id=True)
frame_rate = 5
prev = 0
text_box = (0, 25)
text_color = (255, 0, 0)
font_scale = 1

try:
    while video.isOpened():
        time_elapsed = time.time() - prev
        ret, frame = video.read()
        if time_elapsed > 1./frame_rate:
            prev = time.time()
            if ret:
                begin_time = time.time()
                frame = detecter.predict(frame)
                end_time = time.time()
                cv2.putText(frame,
                            "Mean of time detector: {:.2f}".format(end_time-begin_time),
                            text_box,
                            cv2.FONT_HERSHEY_SIMPLEX,
                            font_scale,
                            text_color,
                            thickness=2,
                            lineType=2)
                cv2.imshow('detector',frame)
            else:
                break
        if cv2.waitKey(20) == ord('q'):
          break

except KeyboardInterrupt:
    pass
except Exception as ex:
    print(ex)
finally:
    video.release()
    cv2.destroyAllWindows()

